# Scrapování větších webů

- Spider, crawler
- Nechci kódovat logiku konkrétní stránky
    - Např. chci prolézt 100 e-shopů
- Jdu jako člověk - po odkazech

## Knihovna Scrapy

Potřebuji napsat:

- URL, na kterých začínám
- Logika, která data ze stránky získat
- Logika, kam pokračovat

Scrapy řeší:

- Jak rychle stahovat
- Již navštívené stránky
- Export dat do různých formátů
- Pamatování cookies
- Organizace většího projektu
    - Rozdělení jednotlivých fází čištění dat
    - Znovupoužití logiky (autentikace, proxy, 

## Scrapy projekt lze exportovat do cloudu

[scrapinghub.com](https://scrapinghub.com)

- Mnoho IP adres
- Více serverů
- Pravidelné scrapování
- Obcházení zákazů

Alternativa v Javascriptu - [apify.com](https://apify.com)

- Sbírka hotových projektů

## Scrapy - ukázka

In [3]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = 'quotes'
    start_urls = [
        'http://quotes.toscrape.com/tag/humor/',
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.xpath('span/small/text()').get(),
            }

        next_page = response.css('li.next a::attr("href")').get()
        if next_page is not None:
            yield response.follow(next_page, self.parse)

## Scrapy - ukázka 2

(Jakoby e-shop)

In [2]:
import scrapy
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor

class MySpider(CrawlSpider):
    name = 'example.com'
    allowed_domains = ['example.com']
    start_urls = ['http://www.example.com']

    rules = (
        # Extract links matching 'category.php' (but not matching 'subsection.php')
        # and follow links from them (since no callback means follow=True by default).
        Rule(LinkExtractor(allow=('category\.php', ), deny=('subsection\.php', ))),

        # Extract links matching 'item.php' and parse them with the spider's method parse_item
        Rule(LinkExtractor(allow=('item\.php', )), callback='parse_item'),
    )

    def parse_item(self, response):
        self.logger.info('Hi, this is an item page! %s', response.url)
        item = scrapy.Item()
        item['id'] = response.xpath('//td[@id="item_id"]/text()').re(r'ID: (\d+)')
        item['name'] = response.xpath('//td[@id="item_name"]/text()').get()
        item['description'] = response.xpath('//td[@id="item_description"]/text()').get()
        item['link_text'] = response.meta['link_text']
        return item